In [1]:
import numpy as np
import pandas as pd
import json
from sklearn.svm import SVR
import pickle
from utils.data_load import Data_load

In [2]:
#读入数据
config = json.load(open('./config.json', 'r'))
V_confirmed = pd.read_csv(config['V_confirmed'], header=None).to_numpy(np.float32)
V_cured = pd.read_csv(config['V_cured'], header=None).to_numpy(np.float32)
V_suspected = pd.read_csv(config['V_suspected'], header=None).to_numpy(np.float32)

In [3]:
V_confirmed = V_confirmed.reshape(V_confirmed.shape[1], V_confirmed.shape[0])

In [76]:
svr_rbf = SVR(kernel='rbf', C=10, gamma=1)

In [48]:
# svr_rbf.fit(X_train, y_train[:,2])

SVR(C=10, gamma=1)

In [27]:
NATree, data_set = Data_load(config, 12, 4, 1, 1, 1)

In [28]:
X_train, y_train = data_set['train']['week'], data_set['train']['target']
X_test, y_test = data_set['test']['week'], data_set['test']['target']

In [29]:
X_train.shape
y_train.shape

(62, 29, 4)

In [30]:
X_train = X_train[:, 2, 0, :]
y_train = y_train[:, 2, :]
X_test = X_test[:, 2, 0, :]
y_test = y_test[:, 2, :]

In [31]:
std = data_set['X_std']
mean = data_set['X_mean']

In [32]:
#声明模型
svr_rbf = SVR(kernel='rbf', C=10, gamma=1)

In [33]:
y_rbf_all = []

In [34]:
#预测未来四天的数据
future = 4
for i in range(future):
   svr_rbf.fit(X_train[:,i+4:i+5], y_train[:,i])
   with open('./results./SVM1/svr{}.pkl'.format(i), 'wb') as f:
       pickle.dump(svr_rbf, f)
   y_rbf = svr_rbf.predict(X_test[:,i+4:i+5])
   y_rbf_all.append(y_rbf)

In [35]:
y_rbf_all = np.array(y_rbf_all)

In [36]:
y_rbf_all = (y_rbf_all*std) + mean

In [37]:
y_rbf_all = np.transpose(y_rbf_all)

In [38]:
y_rbf_df = pd.DataFrame(y_rbf_all)
y_rbf_df.to_csv('./results/SVM1/pred_1.csv',  header=None, index=False)

In [39]:
y_test_tr = (y_test * std) + mean
y_test_tr = pd.DataFrame(y_test_tr)
y_test_tr.to_csv('./results/SVM1/true_1.csv',  header=None, index=False)

In [40]:
#求mae，rmse
val_index = {}
val_index['MAE'] = []
val_index['RMSE'] = []
val_index['MAPE'] = []

In [41]:
y_test_tr = np.array(y_test_tr)

In [42]:
y_rbf_all = np.array(y_rbf_all)

In [43]:
time_slice = [1, 2, 3]
for item in time_slice:
    pred_index = y_rbf_all[item-1]
    val_target_index = y_test_tr[item-1]
    #mae
    mae = np.mean(np.abs(val_target_index - pred_index))
    #remse
    rmse = np.sqrt(np.mean((val_target_index-pred_index)**2))
    #mape
    mape = np.mean(np.abs(val_target_index-pred_index)/(pred_index + 1e-5))
    val_index['MAE'].append(mae)
    val_index['RMSE'].append(rmse)
    val_index['MAPE'].append(mape)

In [132]:
val_index

{'MAE': [2.5635533644652977, 1.8982827560207092, 1.4131006338757288],
 'RMSE': [2.563555910560329, 1.9470059866454865, 1.4582829944264402],
 'MAPE': [0.10915987042045378, 0.07831633144172631, 0.05849931702637394]}

In [133]:
val_index = pd.DataFrame(val_index)

In [134]:

val_index

,MAE,RMSE,MAPE
0,2.563553,2.563556,0.109160
1,1.898283,1.947006,0.078316
2,1.413101,1.458283,0.058499


In [135]:
val_index.to_csv(r'./results/SVM1/mae.csv', index=False, sep=',', encoding='utf-8')